# Hướng dẫn: Cắt chữ ký từ bảng sinh viên
Notebook này giải thích từng bước cách xử lý ảnh bảng để tách chữ ký của từng sinh viên

## 1. Import thư viện cần thiết

In [ ]:
import cv2
import pytesseract
import numpy as np
import os
from matplotlib import pyplot as plt

# Thiết lập để hiển thị ảnh trong notebook
def show_image(img, title='Image', cmap='gray'):
    plt.figure(figsize=(10, 6))
    if len(img.shape) == 3:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

print('✅ Import thành công!')

## 2. Đọc ảnh gốc
Đọc file ảnh bảng có 2 cột: Họ Tên | Chữ ký

In [ ]:
input_file = 'Picture1.png'

# Đọc ảnh
img = cv2.imread(input_file)
if img is None:
    print(f"❌ Không tìm thấy file {input_file}")
else:
    print(f"✅ Đọc file thành công! Kích thước: {img.shape}")
    show_image(img, 'Ảnh gốc')

## 3. Chuyển sang ảnh xám (Grayscale)
Xử lý ảnh xám đơn giản hơn ảnh màu

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
h_img, w_img = gray.shape

print(f"Kích thước ảnh: {w_img}x{h_img} pixels")
show_image(gray, 'Ảnh xám')

## 4. Nhị phân hóa (Thresholding)
Chuyển ảnh thành 2 màu đen trắng để dễ phát hiện đường kẻ bảng

In [ ]:
# Ngưỡng 127: pixel < 127 → đen (255), pixel >= 127 → trắng (0)
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

show_image(thresh, 'Ảnh nhị phân (đảo ngược)')

## 5. Phát hiện đường kẻ NGANG (Horizontal Lines)
Dùng Morphological Operations để tìm các đường kẻ ngang → xác định các hàng trong bảng

In [ ]:
# Tạo kernel ngang (chiều rộng = 1/4 ảnh, chiều cao = 1 pixel)
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (w_img // 4, 1))

# Morphological Opening: loại bỏ nhiễu, giữ lại đường kẻ ngang
detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

show_image(detect_horizontal, 'Đường kẻ ngang')

## 6. Tìm tọa độ Y của các đường kẻ ngang
Quét từng hàng pixel, nếu hàng đó có > 50% pixel đen → đó là đường kẻ

In [ ]:
horizontal_lines = []
for y in range(h_img):
    # Đếm tổng giá trị pixel của hàng y
    if np.sum(detect_horizontal[y, :]) > w_img * 0.5 * 255:
        horizontal_lines.append(y)

print(f"Tìm thấy {len(horizontal_lines)} pixel thuộc đường kẻ ngang")
print(f"Vị trí Y đầu tiên: {horizontal_lines[:10] if len(horizontal_lines) > 10 else horizontal_lines}")

## 7. Gộp các Y gần nhau thành 1 đường kẻ
Đường kẻ có độ dày nhiều pixel → cần gộp lại

In [ ]:
rows_y = []
if horizontal_lines:
    current_y = horizontal_lines[0]
    for y in horizontal_lines[1:]:
        # Nếu cách xa > 10px → đường kẻ mới
        if y - current_y > 10:
            rows_y.append(current_y)
            current_y = y
    rows_y.append(current_y)

print(f"Số đường kẻ ngang (số hàng): {len(rows_y)}")
print(f"Tọa độ Y các đường: {rows_y}")

# Vẽ đường kẻ lên ảnh gốc để kiểm tra
img_with_lines = img.copy()
for y in rows_y:
    cv2.line(img_with_lines, (0, y), (w_img, y), (0, 255, 0), 2)

show_image(img_with_lines, f'Phát hiện {len(rows_y)} đường kẻ ngang')

## 8. Phát hiện đường kẻ DỌC (Vertical Lines)
Tương tự đường ngang, tìm đường dọc ở giữa để chia cột Tên | Chữ ký

In [ ]:
# Kernel dọc (chiều rộng = 1 pixel, chiều cao = 1/4 ảnh)
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, h_img // 4))
detect_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

show_image(detect_vertical, 'Đường kẻ dọc')

# Tìm tọa độ X
vertical_lines = [x for x in range(w_img) if np.sum(detect_vertical[:, x]) > h_img * 0.5 * 255]

# Chọn đường dọc gần giữa nhất
mid_x = min(vertical_lines, key=lambda x: abs(x - w_img // 2)) if vertical_lines else w_img // 2

print(f"Đường dọc giữa tại X = {mid_x}")

# Vẽ đường dọc lên ảnh
img_with_vertical = img.copy()
cv2.line(img_with_vertical, (mid_x, 0), (mid_x, h_img), (255, 0, 0), 2)
show_image(img_with_vertical, f'Đường dọc ở X={mid_x}')

## 9. Cắt từng ô trong bảng
Dựa vào các đường kẻ đã tìm được, cắt ra các ô riêng lẻ

In [ ]:
print(f"Tổng số hàng: {len(rows_y)}")
print(f"Số sinh viên: {len(rows_y) - 1} (bỏ hàng tiêu đề)")

# Hiển thị vài ô mẫu
offset = 1

# Lấy hàng thứ 2 (index=2, bỏ header ở index 0-1)
if len(rows_y) >= 3:
    i = 2
    y_start = rows_y[i - 1]
    y_end = rows_y[i]
    
    # Cắt ô Tên (bên trái)
    roi_name = gray[y_start + offset: y_end - offset, offset: mid_x - offset]
    # Cắt ô Chữ ký (bên phải)
    roi_sig = img[y_start + offset: y_end - offset, mid_x + offset: w_img - offset]
    
    print(f"\nỞ mẫu từ hàng {i}:")
    print(f"  - Vùng Tên: {roi_name.shape}")
    print(f"  - Vùng Chữ ký: {roi_sig.shape}")
    
    # Hiển thị
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].imshow(roi_name, cmap='gray')
    axes[0].set_title('Vùng Tên')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(roi_sig, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Vùng Chữ ký')
    axes[1].axis('off')
    plt.show()

## 10. OCR: Đọc tên sinh viên từ ảnh
Dùng Tesseract OCR để nhận dạng chữ tiếng Việt

In [ ]:
if len(rows_y) >= 3:
    # Tiền xử lý để OCR chính xác hơn
    # 1. Resize lớn hơn (2x)
    roi_name_large = cv2.resize(roi_name, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    
    # 2. Tăng độ tương phản bằng Otsu's thresholding
    roi_name_processed = cv2.threshold(roi_name_large, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    
    show_image(roi_name_processed, 'Ảnh sau tiền xử lý cho OCR')
    
    # OCR với --psm 7 (coi như 1 dòng text)
    name_raw = pytesseract.image_to_string(roi_name_processed, lang='vie', config='--psm 7')
    
    print(f"Tên thô (raw): '{name_raw}'")
    
    # Làm sạch tên (chỉ giữ chữ cái, số, dấu cách)
    name_clean = "".join([c for c in name_raw.strip() if c.isalnum() or c in (' ', '_')]).strip()
    
    print(f"Tên sau khi làm sạch: '{name_clean}'")

## 11. Loại bỏ viền trắng xung quanh chữ ký
Hàm tự động cắt bỏ vùng trắng dư thừa

In [ ]:
def remove_border(img):
    """Loại bỏ viền trắng/đen dư thừa xung quanh ảnh chữ ký"""
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    # Nhị phân hóa: pixel < 200 → nét mực (255), pixel >= 200 → nền trắng (0)
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Tìm tất cả các điểm ảnh có nét mực
    coords = cv2.findNonZero(thresh)
    
    if coords is not None:
        # Tìm hình chữ nhật bao quanh
        x, y, w, h = cv2.boundingRect(coords)
        
        # Thêm padding 10px để không sát mép
        p = 10
        x_start = max(0, x - p)
        y_start = max(0, y - p)
        x_end = min(img.shape[1], x + w + p)
        y_end = min(img.shape[0], y + h + p)
        
        return img[y_start:y_end, x_start:x_end]
    
    return img

# Test hàm
if len(rows_y) >= 3:
    roi_sig_cropped = remove_border(roi_sig)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].imshow(cv2.cvtColor(roi_sig, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Trước khi cắt viền')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(roi_sig_cropped, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Sau khi cắt viền')
    axes[1].axis('off')
    plt.show()
    
    print(f"Kích thước trước: {roi_sig.shape}")
    print(f"Kích thước sau: {roi_sig_cropped.shape}")

## 12. Xử lý TẤT CẢ sinh viên và lưu file
Vòng lặp qua tất cả các hàng, cắt chữ ký và lưu

In [ ]:
output_dir = 'KetQua_ChuKy'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

saved_count = 0

# Bắt đầu từ i=2 để bỏ qua header "Họ Tên | Chữ ký"
for i in range(2, len(rows_y)):
    y_start = rows_y[i - 1]
    y_end = rows_y[i]
    offset = 1
    
    # Cắt ô
    roi_name = gray[y_start + offset: y_end - offset, offset: mid_x - offset]
    roi_sig = img[y_start + offset: y_end - offset, mid_x + offset: w_img - offset]
    
    # OCR
    roi_name_large = cv2.resize(roi_name, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    roi_name_processed = cv2.threshold(roi_name_large, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    name_raw = pytesseract.image_to_string(roi_name_processed, lang='vie', config='--psm 7')
    name_clean = "".join([c for c in name_raw.strip() if c.isalnum() or c in (' ', '_')]).strip()
    
    # Bỏ qua nếu OCR ra rác
    if not name_clean or name_clean in ["Họ Tên", "Chữ ký"]:
        continue
    
    # Cắt viền
    roi_sig_final = remove_border(roi_sig)
    
    # Lưu file (dùng imencode để xử lý tên tiếng Việt)
    save_path = os.path.join(output_dir, f"{name_clean}.png")
    is_success, im_buf_arr = cv2.imencode(".png", roi_sig_final)
    if is_success:
        im_buf_arr.tofile(save_path)
        print(f"✅ Lưu: {save_path}")
        saved_count += 1

print(f"\n🎉 Hoàn tất! Đã lưu {saved_count} chữ ký vào folder '{output_dir}'")

## 13. Xem kết quả cuối cùng

In [ ]:
# Liệt kê các file đã lưu
saved_files = os.listdir(output_dir)
print(f"Các file đã lưu ({len(saved_files)}):")
for f in saved_files:
    print(f"  - {f}")

# Hiển thị tất cả chữ ký đã cắt
if saved_files:
    fig, axes = plt.subplots(1, len(saved_files), figsize=(15, 4))
    if len(saved_files) == 1:
        axes = [axes]
    
    for idx, filename in enumerate(saved_files):
        img_path = os.path.join(output_dir, filename)
        img_sig = cv2.imread(img_path)
        axes[idx].imshow(cv2.cvtColor(img_sig, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(filename.replace('.png', ''), fontsize=10)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

---
## Tổng kết

**Quy trình xử lý:**
1. Đọc ảnh bảng → Chuyển xám
2. Nhị phân hóa ảnh
3. Phát hiện đường kẻ ngang (Morphology) → Tìm các hàng
4. Phát hiện đường kẻ dọc → Tìm ranh giới cột
5. Cắt từng ô riêng lẻ
6. OCR đọc tên (với tiền xử lý)
7. Cắt viền trắng chữ ký
8. Lưu file với tên sinh viên

**Kỹ thuật chính:**
- `cv2.threshold()`: Nhị phân hóa
- `cv2.morphologyEx()`: Phát hiện cấu trúc (đường kẻ)
- `pytesseract`: OCR tiếng Việt
- `cv2.findNonZero()`: Tìm vùng có nội dung